# **class Client**

In [2]:
import torch
import threading
import queue
class Client:
  def __init__(self, client_id,partition1, partition, input_queue=None, output_queue=None, data_loader=None):
        """
        client_id: شناسه کلاینت
        partition: پارتیشن مدل که این کلاینت اجرا می‌کند
        input_queue: صف ورودی برای دریافت داده‌ها
        output_queue: صف خروجی برای ارسال داده‌ها
        data_loader: کلاس DataLoader برای کلاینت اول
        """
        self.client_id = client_id
        self.partition = partition
        self.partition1 = partition1
        self.input_queue = input_queue
        self.output_queue = output_queue
        self.data_loader = data_loader


  def process_partition1(self , data ):
    data_tensor = torch.tensor(data)
    return self.partition1(data_tensor)


  def process_partition(self , data ):
    data_tensor = torch.tensor(data)
    return self.partition(data_tensor)

  def run(self):
    while True:
      input , labels = self.data_loader.get_next_batch()
      output1 = self.process_partition1(inputs)

      if self.input_queue:
        data = self.input_queue.get()
        inputs = torch.tensor(data["output"])
        labels = data["labels"]
        labels = self.process_partition1(inputs)
      output2 = self.process_partition(output1)

# **Data Loader Class**

In [13]:
import random
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader as TorchDataLoader

class DataLoader:
    def __init__(self, dataset, batch_size=32, shuffle=True, train_ratio=0.8):
        """
        Initialize the DataLoader with specific configurations.
        """
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.train_ratio = train_ratio

        self.data_index = 0
        self.mode = "train"

        # set transform
        self.transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

        self.dataset_train , self.dataset_test = self.loadDataSet()

        # create loader for test and train
        self.train_loader = TorchDataLoader(self.dataset_train , batch_size=self.batch_size , shuffle=self.shuffle)
        self.test_loader = TorchDataLoader(self.dataset_test , batch_size=self.batch_size , shuffle=False)
        self.mode = "train"

    def set_mode(self, mode):
        """
        train and test mod
        """
        if mode not in ["train", "test"]:
            raise ValueError("Mode must be either 'train' or 'test'")
        self.mode = mode

    def loadDataSet(self):
      trainset = torchvision.datasets.CIFAR10(root='./data',train=True ,download = True ,transform= self.transform)
      testset = torchvision.datasets.CIFAR10(root='./data', train=False ,download = True , transform=self.transform )
      return trainset , testset


    def getLoader(self):
      if self.mode == "train":
        return self.train_loader
      else:
        return self.test_loader

    def get_next_batch(self):
      loader = self.getLoader()
      return next(iter(loader))

# **define Partion class**

In [4]:
class Partition:
    def __init__(self, in_channels, out_channels, num_blocks, stride=1):
        """
        Base class for a partition with configurable architecture.
        """
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.num_blocks = num_blocks
        self.stride = stride
        self.build_model()

In [5]:
class Partition1(Partition):
    def build_model(self):
        # Define specific layers for Partition1
        pass

# **Message Class**

In [6]:
class Message:
  def __init__(self , msg_type , stage, source , destination,data_id,data):
    self.msg_type = msg_type
    self.stage = stage
    self.source = source
    self.destination = destination
    self.data_id = data_id
    self.data = data

# **test DataLoader Class**

In [16]:
# ایجاد یک شیء از DataLoader
data_loader = DataLoader(dataset='CIFAR10',batch_size=64, shuffle=True)

# تنظیم حالت به آموزش
data_loader.set_mode("train")

# دریافت batch بعدی از داده‌های آموزشی
inputs, labels = data_loader.get_next_batch()
print(f"Batch shape: {inputs.shape}, Labels shape: {labels.shape}")

# تغییر حالت به تست
data_loader.set_mode("test")
inputs, labels = data_loader.get_next_batch()
print(f"Batch shape: {inputs.shape}, Labels shape: {labels.shape}")


Files already downloaded and verified
Files already downloaded and verified
Batch shape: torch.Size([64, 3, 32, 32]), Labels shape: torch.Size([64])
Batch shape: torch.Size([64, 3, 32, 32]), Labels shape: torch.Size([64])
